# CartPole Gym environment with TfAgents

## Install packages (gym, tfagents, tensorflow,....)

In [2]:
!pip install easyagents >/dev/null

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
#!pip install -i https://test.pypi.org/simple/ easyagents > /dev/null

## Dry Run (very short training, no logging)

In [6]:
from easyagents.tfagents import PpoAgent
from easyagents.config import TrainingDurationFast
from easyagents.config import LoggingSilent
from easyagents.config import LoggingVerbose

ppoAgent = PpoAgent( gym_env_name='CartPole-v0', training_duration=TrainingDurationFast(), logging=LoggingSilent())
ppoAgent.train()

W0626 21:48:11.276243 13692 deprecation.py:323] From c:\dev\github\easyagents\env\lib\site-packages\tensorflow\python\ops\math_grad.py:1221: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## Visualize

In [ ]:
ppoAgent.plot_average_returns()

In [ ]:
ppoAgent.plot_losses()